In [7]:
import os 
import glob 
import shutil 
import pandas as pd 
 
 
# Ensure backup folder exists
if not os.path.exists("backup_folder"):
    os.makedirs("backup_folder")
     
# Move all CSV files to a backup folder 
csv_files = glob.glob("csv_files/*.csv") 
for file in csv_files: 
    shutil.move(file, "backup_folder/") 
    print(f"Moved file: {file}") 
 
# Automating Export 
def export_data(df, filename, format): 
    if format == "csv": 
        df.to_csv(filename, index=False) 
        print(f"Data exported to {filename} in CSV format.") 
    elif format == "json": 
        df.to_json(filename, orient="records") 
        print(f"Data exported to {filename} in JSON format.") 
    else: 
        print("Unsupported format.")          
    
 
# Example usage: 
# Creating a sample dataframe 
data = {'Name': ['Alice', 'Bob', 'Charlie'], 
        'Age': [25, 30, 35], 
        'City': ['New York', 'Los Angeles', 'Chicago']} 
 
df = pd.DataFrame(data) 
 
# Exporting to CSV 
export_data(df, "output.csv", "csv") 
 
# Exporting to JSON 
export_data(df, "output.json", "json")


Moved file: csv_files/data3.csv
Moved file: csv_files/data2.csv
Moved file: csv_files/data.csv
Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.


In [1]:
import yfinance as yf
import sqlite3
import pandas as pd
import time

# Database setup
db_name = "stocks.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                symbol TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume INTEGER)''')
conn.commit()

# Function to fetch stock data
def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1m")

        if data.empty:
            print(f"No data found for {symbol}. Skipping...")
            return None

        latest = data.iloc[-1]  # Get the most recent price data
        return {
            "symbol": symbol,
            "open": latest["Open"],
            "high": latest["High"],
            "low": latest["Low"],
            "close": latest["Close"],
            "volume": latest["Volume"]
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to store data in SQLite
def store_data(symbol):
    stock_data = fetch_stock_data(symbol)
    if stock_data:
        cursor.execute('''INSERT INTO stock_data (symbol, open, high, low, close, volume)
                          VALUES (?, ?, ?, ?, ?, ?)''',
                       (stock_data["symbol"], stock_data["open"],
                        stock_data["high"], stock_data["low"],
                        stock_data["close"], stock_data["volume"]))
        conn.commit()
        print(f"Stored data for {symbol}")

# Function to analyze stock data
def analyze_stock(symbol):
    df = pd.read_sql_query("SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100", conn, params=(symbol,))
    print(df)

# Example Usage
symbol = "GOOGL"  
for _ in range(5):  # Fetch data 5 times with intervals
    store_data(symbol)
    time.sleep(60)  # Wait for 1 minute before fetching again

analyze_stock(symbol)

# Close database connection
conn.close()


/Users/maheen/Documents/ITTrainingByPlusW/.venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Stored data for GOOGL
Stored data for GOOGL
Stored data for GOOGL
Stored data for GOOGL
Stored data for GOOGL
   id symbol            timestamp        open        high         low  \
0   5  GOOGL  2025-02-23 06:49:04  179.634995  179.710007  179.479996   
1   4  GOOGL  2025-02-23 06:48:03  179.634995  179.710007  179.479996   
2   3  GOOGL  2025-02-23 06:47:02  179.634995  179.710007  179.479996   
3   2  GOOGL  2025-02-23 06:45:59  179.634995  179.710007  179.479996   
4   1  GOOGL  2025-02-23 06:44:58  179.634995  179.710007  179.479996   

        close  volume  
0  179.660004  652240  
1  179.660004  652240  
2  179.660004  652240  
3  179.660004  652240  
4  179.660004  652240  


In [2]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd 
 
URL = "http://books.toscrape.com/" 
HEADERS = {"User-Agent": "Mozilla/5.0"} 
 
def get_books(url): 
    response = requests.get(url, headers=HEADERS) 
    soup = BeautifulSoup(response.text, "html.parser") 
     
    books = soup.find_all("article", class_="product_pod") 
    book_list = [] 
 
    for book in books: 
        title = book.h3.a["title"] 
        price = book.find("p", class_="price_color").text 
        stock = book.find("p", class_="instock availability").text.strip() 
 
        book_list.append({"Title": title, "Price": price, "Availability": 
                            stock}) 
 
    return book_list 
 
books_data = get_books(URL) 
df = pd.DataFrame(books_data) 
df.to_csv("books.csv", index=False) 
print("Data saved to books.csv")

Data saved to books.csv
